# **Importing the Certain Libraries**

In [1]:
import numpy as np
import pandas as pd
import preprocessor as p
import collections
import os
import re
import string
import time
import warnings
from collections import Counter
from string import punctuation

#Visualisation
from wordcloud import WordCloud, ImageColorGenerator, STOPWORDS
from PIL import Image
import matplotlib.pyplot as plt
import plotly.offline as py
import plotly.graph_objs as go
import matplotlib
import seaborn as sns

#nltk
from nltk import PorterStemmer
from nltk import FreqDist
from nltk.collocations import *
from nltk.stem import WordNetLemmatizer
from nltk.sentiment.vader import SentimentIntensityAnalyzer
from nltk.sentiment.util import *
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import PunktSentenceTokenizer

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

from keras.preprocessing.text import Tokenizer
from keras.preprocessing import sequence
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Bidirectional
from keras.layers import Embedding, Conv1D, Flatten, MaxPooling1D
from keras.layers import Activation, Dense, Dropout, LSTM
from keras.optimizers import Adam
from keras.utils import to_categorical

from tqdm import tqdm
tqdm.pandas(desc="progress-bar")

warnings.filterwarnings("ignore", category=DeprecationWarning)

plt.style.use('fivethirtyeight')
%matplotlib inline
%config IPCompleter.greedy=True

C:\Users\Arunava\Anaconda3\lib\site-packages\statsmodels\tools\_testing.py:19: FutureWarning:

pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.



In [2]:
import tensorflow as tf
config = tf.compat.v1.ConfigProto()
config.gpu_options.allow_growth = True
sess = tf.compat.v1.Session(config=config)

In [3]:
original_df = pd.read_csv('TweetsDB.csv')
print(original_df.shape)
original_df.head(5)

(1710, 12)


,id,created_at,source,original_text,clean_text,lang,favorite_count,retweet_count,original_author,hashtags,user_mentions,place
0,1.250000e+18,Sun Apr 19 21:21:54 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @GlblCtzn: .@priyankachopra is calling on l...,calling leaders help protect refugees COVID-19...,en,0,31,RJIshak,NaN,"GlblCtzn, priyankachopra",Jakarta Capital Region
1,1.250000e+18,Sun Apr 19 21:21:53 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,OGUN STATE SUPPORT FOR CBN-NIRSAL COVID-19 TAR...,en,0,61,makinwaoluwole,NaN,OGSG_Official,Nigeria
2,1.250000e+18,Sun Apr 19 21:21:52 +0000 2020,"<a href=""http://twitter.com/download/iphone"" r...",RT @AdvoBarryRoux: These 5 police officials ba...,These police officials based Namahadi police s...,en,0,1,TembeAmu,NaN,AdvoBarryRoux,NaN
3,1.250000e+18,Sun Apr 19 21:21:51 +0000 2020,"<a href=""http://twitter.com/download/iphone"" r...",RT @MobilePunch: COVID-19: Oyo discharges two ...,COVID-19 Oyo discharges two patients,en,0,0,ilyasrabiu,NaN,MobilePunch,"Lagos, Nigeria"
4,1.250000e+18,Sun Apr 19 21:21:50 +0000 2020,"<a href=""http://twitter.com/download/android"" ...",My Condolences to the Family of those who did ...,My Condolences Family survive,en,0,13869,bucketeconomist,Covid_19,NaN,NaN


In [4]:
# dropping ALL duplicte values
original_df.drop_duplicates(subset ="original_text", keep = "first", inplace = True)

# changing the date format
original_df['created_at'] = pd.to_datetime(original_df['created_at'], format='%a %b %d %H:%M:%S %z %Y').dt.strftime('%Y-%m-%d')

original_df.to_csv("TweetsDB.csv", index=False)

original_df = pd.read_csv('TweetsDB.csv')
new_full_df = original_df
print(new_full_df.shape)
new_full_df.head(5)

(1587, 12)


,id,created_at,source,original_text,clean_text,lang,favorite_count,retweet_count,original_author,hashtags,user_mentions,place
0,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @GlblCtzn: .@priyankachopra is calling on l...,calling leaders help protect refugees COVID-19...,en,0,31,RJIshak,NaN,"GlblCtzn, priyankachopra",Jakarta Capital Region
1,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,OGUN STATE SUPPORT FOR CBN-NIRSAL COVID-19 TAR...,en,0,61,makinwaoluwole,NaN,OGSG_Official,Nigeria
2,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @AdvoBarryRoux: These 5 police officials ba...,These police officials based Namahadi police s...,en,0,1,TembeAmu,NaN,AdvoBarryRoux,NaN
3,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @MobilePunch: COVID-19: Oyo discharges two ...,COVID-19 Oyo discharges two patients,en,0,0,ilyasrabiu,NaN,MobilePunch,"Lagos, Nigeria"
4,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",My Condolences to the Family of those who did ...,My Condolences Family survive,en,0,13869,bucketeconomist,Covid_19,NaN,NaN


# **Frequency of Tweets**

In [5]:
freq_tweet_df = pd.DataFrame()
freq_tweet_df["Date"] = new_full_df['created_at']
freq_tweet_df["Tweets_Freq"] = new_full_df['original_text']
freq_tweet_df = freq_tweet_df.groupby('Date')['Tweets_Freq'].count().reset_index()
freq_tweet_df.head(5)

,Date,Tweets_Freq
0,2020-04-19,28
1,2020-04-22,30
2,2020-04-23,28
3,2020-04-24,29
4,2020-04-25,30


# **Preprocessing of Raw Data**

In [6]:
# remove urls, RT, handles, and the hashtag from hashtags
def remove_urls(text):
    new_text = re.sub(r'amp;', '', str(text))
    new_text = re.sub(r':', '', str(new_text))
    new_text = re.sub(r'‚Ä¶', '', str(new_text))
    #replace consecutive non-ASCII characters with a space
    new_text = re.sub(r'[^\x00-\x7F]+',' ', str(new_text))
    new_text = ' '.join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",str(new_text)).split())
    return new_text

# make all text lowercase
def text_lowercase(text):
    return text.lower()

#remove numbers
def remove_numbers(text):
    result = re.sub(r'\d+', '', str(text))
    return result

# remove punctuation
def remove_punctuation(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

# tokenize
def tokenize(text):
    text = word_tokenize(text)
    return text

# remove stopwords
stop_words = set(stopwords.words('english'))
def remove_stopwords(text):
    text = [i for i in text if not i in stop_words]
    return text

# lemmatize
lemmatizer = WordNetLemmatizer()
def lemmatize(text):
    text = [lemmatizer.lemmatize(token) for token in text]
    return text

def preprocessing(text):
    text = remove_urls(text)
    text = text_lowercase(text)
    text = remove_numbers(text)
    text = remove_punctuation(text)
    text = tokenize(text)
    text = remove_stopwords(text)
    text = lemmatize(text)
    text = ' '.join(text)
    text = re.sub(r'covid', 'covid19', str(text))
    return text

In [7]:
clean_text = [] # our preprocessed text column
for text_data in new_full_df['original_text']:
    pp_text_data = p.clean(str(text_data))
    clean_text_data = preprocessing(pp_text_data)
    clean_text.append(clean_text_data)

new_full_df['clean_tweet'] = clean_text # add the preprocessed text as a column
new_full_df.head(5)

,id,created_at,source,original_text,clean_text,lang,favorite_count,retweet_count,original_author,hashtags,user_mentions,place,clean_tweet
0,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @GlblCtzn: .@priyankachopra is calling on l...,calling leaders help protect refugees COVID-19...,en,0,31,RJIshak,NaN,"GlblCtzn, priyankachopra",Jakarta Capital Region,calling leader help protect refugee covid19 pr...
1,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,OGUN STATE SUPPORT FOR CBN-NIRSAL COVID-19 TAR...,en,0,61,makinwaoluwole,NaN,OGSG_Official,Nigeria,ogun state support cbn nirsal covid19 targeted...
2,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @AdvoBarryRoux: These 5 police officials ba...,These police officials based Namahadi police s...,en,0,1,TembeAmu,NaN,AdvoBarryRoux,NaN,police official based namahadi police station ...
3,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @MobilePunch: COVID-19: Oyo discharges two ...,COVID-19 Oyo discharges two patients,en,0,0,ilyasrabiu,NaN,MobilePunch,"Lagos, Nigeria",covid19 oyo discharge two patient
4,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",My Condolences to the Family of those who did ...,My Condolences Family survive,en,0,13869,bucketeconomist,Covid_19,NaN,NaN,condolence family survive


In [8]:
tokenized_tweet = new_full_df['clean_tweet'].apply(lambda x: x.split())
tokenized_tweet.head()

0    [calling, leader, help, protect, refugee, covi...
1    [ogun, state, support, cbn, nirsal, covid19, t...
2    [police, official, based, namahadi, police, st...
3              [covid19, oyo, discharge, two, patient]
4                        [condolence, family, survive]
Name: clean_tweet, dtype: object

In [9]:
ps = PorterStemmer()
Stemmed_tweet = tokenized_tweet.apply(lambda x: [ps.stem(i) for i in x])
Stemmed_tweet.head()

0    [call, leader, help, protect, refuge, covid19,...
1    [ogun, state, support, cbn, nirsal, covid19, t...
2    [polic, offici, base, namahadi, polic, station...
3               [covid19, oyo, discharg, two, patient]
4                             [condol, famili, surviv]
Name: clean_tweet, dtype: object

In [10]:
for i in range(len(Stemmed_tweet)):
    Stemmed_tweet[i] = ' '.join(Stemmed_tweet[i])

new_full_df['clean_tweet'] = Stemmed_tweet
new_full_df['clean_tweet'].replace('', np.nan, inplace=True)
new_full_df['clean_tweet'] = new_full_df['clean_tweet'].str.replace(r'\b\w\b', '').str.replace(r'\s+', ' ')
new_full_df.head(5)

,id,created_at,source,original_text,clean_text,lang,favorite_count,retweet_count,original_author,hashtags,user_mentions,place,clean_tweet
0,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @GlblCtzn: .@priyankachopra is calling on l...,calling leaders help protect refugees COVID-19...,en,0,31,RJIshak,NaN,"GlblCtzn, priyankachopra",Jakarta Capital Region,call leader help protect refuge covid19 provid...
1,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,OGUN STATE SUPPORT FOR CBN-NIRSAL COVID-19 TAR...,en,0,61,makinwaoluwole,NaN,OGSG_Official,Nigeria,ogun state support cbn nirsal covid19 target c...
2,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @AdvoBarryRoux: These 5 police officials ba...,These police officials based Namahadi police s...,en,0,1,TembeAmu,NaN,AdvoBarryRoux,NaN,polic offici base namahadi polic station busi ...
3,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/iphone"" r...",RT @MobilePunch: COVID-19: Oyo discharges two ...,COVID-19 Oyo discharges two patients,en,0,0,ilyasrabiu,NaN,MobilePunch,"Lagos, Nigeria",covid19 oyo discharg two patient
4,1.250000e+18,2020-04-19,"<a href=""http://twitter.com/download/android"" ...",My Condolences to the Family of those who did ...,My Condolences Family survive,en,0,13869,bucketeconomist,Covid_19,NaN,NaN,condol famili surviv


## **Save the Preprocessed tweets into Original Dataset**

In [11]:
df = pd.read_csv('TweetsDB.csv')
df["clean_tweet"] = new_full_df['clean_tweet']
df.to_csv("TweetsDB.csv", index=False)
new_full_df = pd.read_csv('TweetsDB.csv')

# **Feature I: Covid-19 Specified Words Identification**

## **Filter Words**

In [12]:
all_words = ' '.join(str(text) for text in new_full_df['clean_tweet'])
# split into words
tokens = word_tokenize(all_words)
word_dist = nltk.FreqDist(tokens)

# convert to lower case
tokens = [w.lower() for w in tokens]

# remove punctuation from each word
table = str.maketrans('', '', string.punctuation)
stripped = [w.translate(table) for w in tokens]

# remove remaining tokens that are not alphabetic
# words = [word for word in stripped if word.isalpha()]

# filter out stop words
# stop_words = set(stopwords.words('english'))
# words = [w for w in words if not w in stop_words]

print(tokens[0:], file=open("Covid Tokens.txt", "w"))
print(tokens[0:])

['call', 'leader', 'help', 'protect', 'refuge', 'covid19', 'provid', 'qualiti', 'health', 'care', 'ogun', 'state', 'support', 'cbn', 'nirsal', 'covid19', 'target', 'credit', 'facil', 'tcf', 'polic', 'offici', 'base', 'namahadi', 'polic', 'station', 'busi', 'drink', 'liquor', 'certain', 'tavern', 'whilst', 'duti', 'covid19', 'oyo', 'discharg', 'two', 'patient', 'condol', 'famili', 'surviv', 'receiv', 'text', 'year', 'old', 'son', 'work', 'covid19', 'patient', 'equat', 'quarantin', 'com', 'taiwan', 'vice', 'presid', 'chen', 'chien', 'jen', 'countri', 'fight', 'covid19', 'break', 'new', 'york', 'woman', 'symptom', 'die', 'last', 'week', 'prescrib', 'drug', 'cocktail', 'known', 'horribl', 'tragedi', 'nova', 'scotia', 'today', 'famili', 'get', 'closur', 'pandem', 'funer', 'covid19', 'cure', 'sooner', 'later', 'cure', 'ghose', 'viru', 'januari', 'nd', 'total', 'control', 'one', 'person', 'come', 'china', 'control', 'go', 'doctor', 'friend', 'deliv', 'babi', 'crisi', 'told', 'interest', 'data

## **Covid-19 Specified Words Popularity**

In [13]:
# Create counter
word_counts = collections.Counter(tokens)
word_counts.most_common()

word_counts_df = pd.DataFrame(word_counts.most_common(),
                             columns=['Word', 'Popularity'])
word_counts_df.head(5)

,Word,Popularity
0,covid19,912
1,case,122
2,new,119
3,peopl,118
4,test,112


In [14]:
probability = []
total_words_count = word_counts_df['Popularity'].sum()

for i in range(0,len(word_counts_df.index)):
    word_probability = word_counts_df['Popularity'].iloc[i] / total_words_count
    probability.append(word_probability)

word_counts_df['Probability'] = probability
word_counts_df = word_counts_df.dropna()
word_counts_df.to_csv("Word Frequency.csv", index=False)
word_counts_df.head(5)

,Word,Popularity,Probability
0,covid19,912,0.057975
1,case,122,0.007755
2,new,119,0.007565
3,peopl,118,0.007501
4,test,112,0.007120


In [15]:
word = []
popularity = []
probability = []

specific_word_df = pd.DataFrame()
specific_word_df["Word"] = ""
specific_word_df["Popularity"] = ""
specific_word_df["Probability"] = ""

for i in range(0,len(word_counts_df.index)):
    if word_counts_df['Probability'].iloc[i] >= 0.000001:
        word.append(word_counts_df['Word'].iloc[i])
        popularity.append(word_counts_df['Popularity'].iloc[i])
        probability.append(word_counts_df['Probability'].iloc[i])

specific_word_df['Word'] = word
specific_word_df['Popularity'] = popularity
specific_word_df['Probability'] = probability
specific_word_df = specific_word_df.dropna()
specific_word_df.to_csv("Covid-19 Specific Words.csv", index=False)
specific_word_df.head(5)

,Word,Popularity,Probability
0,covid19,912,0.057975
1,case,122,0.007755
2,new,119,0.007565
3,peopl,118,0.007501
4,test,112,0.007120


# **Feature II: Word Popularity Detection using N-gram**

## **Unigram Analysis**

In [16]:
ug_count = Counter(tokens).most_common()
print(ug_count, file=open("Unigram Analysis.txt", "w"))
print(ug_count)

[('covid19', 912), ('case', 122), ('new', 119), ('peopl', 118), ('test', 112), ('death', 86), ('pandem', 76), ('day', 71), ('say', 68), ('health', 67), ('trump', 65), ('today', 61), ('die', 59), ('work', 55), ('patient', 54), ('help', 53), ('one', 49), ('go', 49), ('home', 49), ('report', 48), ('time', 48), ('may', 48), ('state', 47), ('take', 47), ('govern', 47), ('due', 46), ('get', 45), ('like', 43), ('worker', 41), ('week', 40), ('need', 40), ('use', 39), ('coronaviru', 39), ('posit', 38), ('world', 37), ('news', 35), ('mani', 34), ('infect', 34), ('confirm', 34), ('lockdown', 34), ('support', 33), ('doctor', 33), ('nation', 33), ('updat', 33), ('hospit', 32), ('respons', 32), ('life', 32), ('care', 31), ('look', 31), ('would', 31), ('commun', 31), ('number', 30), ('mask', 29), ('make', 29), ('call', 28), ('presid', 28), ('countri', 28), ('see', 28), ('person', 27), ('china', 27), ('know', 27), ('want', 27), ('american', 27), ('year', 26), ('uk', 26), ('even', 26), ('citi', 26), ('

## **Bigram Analysis**

In [17]:
bigram_measures = nltk.collocations.BigramAssocMeasures()
finder = BigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(3)
print(finder.nbest(bigram_measures.pmi, 100), file=open("Bigram Analysis.txt", "w"))
print(finder.nbest(bigram_measures.pmi, 100))

[('trevor', 'bell'), ('suprem', 'court'), ('georg', 'floyd'), ('hong', 'kong'), ('domin', 'cum'), ('taxi', 'driver'), ('bill', 'gate'), ('front', 'line'), ('pay', 'tribut'), ('climat', 'chang'), ('infecti', 'diseas'), ('per', 'caput'), ('sign', 'petit'), ('second', 'wave'), ('social', 'distanc'), ('prime', 'minist'), ('million', 'dollar'), ('ralli', 'tulsa'), ('lift', 'restrict'), ('flu', 'season'), ('best', 'wish'), ('donat', 'plasma'), ('last', 'night'), ('great', 'sad'), ('mortal', 'rate'), ('pass', 'away'), ('south', 'africa'), ('year', 'old'), ('polit', 'leader'), ('contact', 'trace'), ('white', 'hous'), ('wear', 'mask'), ('mr', 'presid'), ('sad', 'announc'), ('data', 'scientist'), ('releas', 'earli'), ('staff', 'member'), ('low', 'risk'), ('work', 'tirelessli'), ('discharg', 'hospit'), ('record', 'nigeria'), ('th', 'june'), ('bihar', 'take'), ('month', 'ago'), ('donald', 'trump'), ('measur', 'place'), ('unit', 'state'), ('make', 'sure'), ('around', 'world'), ('stay', 'safe'), ('i

In [18]:
bgs = nltk.bigrams(tokens)

#compute frequency distribution for all the bigrams in the text
fdist = nltk.FreqDist(bgs)
bg_count = Counter(fdist).most_common()
print(bg_count, file=open("Bigram Analysis.txt", "w"))
print(bg_count)

[(('covid19', 'case'), 44), (('covid19', 'pandem'), 39), (('due', 'covid19'), 33), (('covid19', 'test'), 30), (('covid19', 'patient'), 28), (('covid19', 'death'), 27), (('test', 'posit'), 25), (('die', 'covid19'), 24), (('new', 'case'), 21), (('posit', 'covid19'), 19), (('case', 'covid19'), 19), (('new', 'covid19'), 18), (('peopl', 'die'), 15), (('respons', 'covid19'), 13), (('nurs', 'home'), 12), (('recov', 'covid19'), 11), (('wear', 'mask'), 11), (('year', 'old'), 10), (('death', 'covid19'), 10), (('covid19', 'crisi'), 10), (('spread', 'covid19'), 10), (('covid19', 'infect'), 10), (('new', 'york'), 9), (('social', 'distanc'), 9), (('cure', 'covid19'), 9), (('test', 'covid19'), 9), (('fight', 'covid19'), 8), (('covid19', 'covid19'), 8), (('covid19', 'updat'), 8), (('world', 'health'), 8), (('death', 'toll'), 8), (('mental', 'health'), 8), (('covid19', 'lockdown'), 8), (('confirm', 'case'), 8), (('look', 'like'), 8), (('health', 'worker'), 8), (('use', 'covid19'), 7), (('covid19', 'imp

## **Trigram Analysis**

In [19]:
trigram_measures = nltk.collocations.TrigramAssocMeasures()
finder = TrigramCollocationFinder.from_words(tokens)
finder.apply_freq_filter(3)
print(finder.nbest(trigram_measures.pmi, 100))

[('great', 'sad', 'announc'), ('bihar', 'take', 'total'), ('world', 'health', 'organ'), ('case', 'bihar', 'take'), ('death', 'per', 'million'), ('peopl', 'lost', 'life'), ('second', 'wave', 'covid19'), ('covid19', 'record', 'nigeria'), ('covid19', 'infect', 'discharg'), ('sadli', 'die', 'covid19'), ('report', 'new', 'case'), ('covid19', 'case', 'bihar'), ('covid19', 'death', 'toll'), ('total', 'number', 'covid19'), ('test', 'posit', 'covid19'), ('look', 'like', 'covid19'), ('covid19', 'death', 'per'), ('updat', 'day', 'covid19'), ('new', 'covid19', 'case'), ('case', 'covid19', 'record'), ('confirm', 'new', 'covid19'), ('case', 'covid19', 'infect'), ('new', 'case', 'covid19'), ('get', 'test', 'covid19'), ('peopl', 'die', 'covid19'), ('day', 'covid19', 'case')]


In [20]:
bgs = nltk.trigrams(tokens)

#compute frequency distribution for all the trigrams in the text
fdist = nltk.FreqDist(bgs)
tg_count = Counter(fdist).most_common()
print(tg_count, file=open("Trigram Analysis.txt", "w"))
print(tg_count)

[(('test', 'posit', 'covid19'), 18), (('new', 'covid19', 'case'), 15), (('new', 'case', 'covid19'), 9), (('world', 'health', 'organ'), 5), (('report', 'new', 'case'), 5), (('covid19', 'death', 'toll'), 4), (('peopl', 'die', 'covid19'), 4), (('great', 'sad', 'announc'), 3), (('sadli', 'die', 'covid19'), 3), (('updat', 'day', 'covid19'), 3), (('day', 'covid19', 'case'), 3), (('covid19', 'case', 'bihar'), 3), (('case', 'bihar', 'take'), 3), (('bihar', 'take', 'total'), 3), (('death', 'per', 'million'), 3), (('case', 'covid19', 'infect'), 3), (('total', 'number', 'covid19'), 3), (('covid19', 'infect', 'discharg'), 3), (('peopl', 'lost', 'life'), 3), (('look', 'like', 'covid19'), 3), (('covid19', 'death', 'per'), 3), (('confirm', 'new', 'covid19'), 3), (('second', 'wave', 'covid19'), 3), (('get', 'test', 'covid19'), 3), (('case', 'covid19', 'record'), 3), (('covid19', 'record', 'nigeria'), 3), (('record', 'first', 'covid19'), 2), (('peopl', 'kill', 'day'), 2), (('maintain', 'social', 'dista

# **Sentiment Analysis on the Clean Tweets**

## **Finding Sentiment Polarity for each Tweets**

In [21]:
def nltk_sentiment(sentence):
    nltk_sentiment = SentimentIntensityAnalyzer()
    score = nltk_sentiment.polarity_scores(sentence)
    return score

In [22]:
dataset = new_full_df['clean_tweet'].astype(str)
nltk_results = [nltk_sentiment(row) for row in dataset]
results_df = pd.DataFrame(nltk_results)
text_df = pd.DataFrame(dataset)
sentiment_df = text_df.join(results_df)

## **Sentimet Classification**

In [23]:
pos_count, neg_count, neu_count, sum_count = 0, 0, 0, 0
per_pos_count, per_neg_count, per_neu_count = 0.0, 0.0, 0.0

for i in range(0,len(sentiment_df.index)):
    if sentiment_df['compound'].iloc[i] < 0:
        neg_count += 1
    elif sentiment_df['compound'].iloc[i] > 0:
        pos_count += 1
    else:
        neu_count += 1

sum_count = neg_count + pos_count + neu_count
per_pos_count = (pos_count/sum_count)*100
per_neg_count = (neg_count/sum_count)*100
per_neu_count = (neu_count/sum_count)*100

print("The Possitive tweets found : ", round(per_pos_count,2), "%")
print("The Negetive tweets found : ", round(per_neg_count,2), "%")
print("The Neutral tweets found : ", round(per_neu_count,2), "%")

The Possitive tweets found :  29.8 %
The Negetive tweets found :  31.7 %
The Neutral tweets found :  38.5 %


In [24]:
sentiment_df["date"] = original_df['created_at']
sentiment_df.head(5)

,clean_tweet,neg,neu,pos,compound,date
0,call leader help protect refuge covid19 provid...,0.0,0.452,0.548,0.8176,2020-04-19
1,ogun state support cbn nirsal covid19 target c...,0.0,0.602,0.398,0.6486,2020-04-19
2,polic offici base namahadi polic station busi ...,0.0,0.851,0.149,0.2732,2020-04-19
3,covid19 oyo discharg two patient,0.0,1.000,0.000,0.0000,2020-04-19
4,condol famili surviv,0.0,1.000,0.000,0.0000,2020-04-19


In [25]:
sentiment_df["sentiment"] = ""

for i in range(0,len(sentiment_df.index)):
    if sentiment_df['compound'].iloc[i] < 0:
        sentiment_df['sentiment'].iloc[i] = 'neg'
    elif sentiment_df['compound'].iloc[i] > 0:
        sentiment_df['sentiment'].iloc[i] = 'pos'
    else:
        sentiment_df['sentiment'].iloc[i] = 'neu'

C:\Users\Arunava\Anaconda3\lib\site-packages\pandas\core\indexing.py:671: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [26]:
new_df = pd.DataFrame()
new_df = pd.read_csv('TweetsDB.csv')
new_df[["compound", "neg", "neu", "pos", "sentiment"]] = sentiment_df[['compound', 'neg', 'neu', 'pos', 'sentiment']]
new_df.to_csv("TweetsDB.csv", index=False)

In [27]:
cols = ['Date', 'Text', 'Sentiment']
new_sentiment_df = pd.DataFrame(columns=cols, index=range(len(sentiment_df.index)))
for i in range(0,len(sentiment_df.index)):
    if (sentiment_df.iloc[i]['sentiment'] != 'neu'):
        new_sentiment_df.loc[i].Date = sentiment_df.at[i, 'date']
        new_sentiment_df.loc[i].Text = sentiment_df.at[i, 'clean_tweet']
        new_sentiment_df.loc[i].Sentiment = sentiment_df.at[i, 'sentiment']
new_sentiment_df = new_sentiment_df.dropna()
new_sentiment_df = new_sentiment_df.reset_index(drop=True)
new_sentiment_df = new_sentiment_df
data_df = pd.DataFrame()
data_df[['Sentiment', 'Text']] = new_sentiment_df[['Sentiment', 'Text']]
data_df.to_csv("Data.csv", index=False)
data_df.head(5)

,Sentiment,Text
0,pos,call leader help protect refuge covid19 provid...
1,pos,ogun state support cbn nirsal covid19 target c...
2,pos,polic offici base namahadi polic station busi ...
3,neg,taiwan vice presid chen chien jen countri figh...
4,neg,break new york woman symptom die last week pre...


In [28]:
tweet_df = pd.DataFrame()
tweet_df["Text"] = data_df['Text']
tweet_df.to_csv("Tweet Data.csv", index=False)
tweet_df.head(5)

,Text
0,call leader help protect refuge covid19 provid...
1,ogun state support cbn nirsal covid19 target c...
2,polic offici base namahadi polic station busi ...
3,taiwan vice presid chen chien jen countri figh...
4,break new york woman symptom die last week pre...


# **Naïve Bayes Sentiment Model**

In [29]:
tweets = pd.read_csv('Tweet Data.csv', header=0)
data = pd.read_csv('Data.csv', sep=',', names=['Sentiment', 'Text'], dtype=str, header=0)

def split_into_lemmas(tweet):
    ngram_vectorizer = CountVectorizer(ngram_range=(1, 3), token_pattern=r'\b\w+\b', min_df=1)
    analyze = ngram_vectorizer.build_analyzer()
    return analyze(tweet)

bow_transformer = CountVectorizer(analyzer=split_into_lemmas, stop_words='english', strip_accents='ascii').fit(data['Text'])

text_bow = bow_transformer.transform(data['Text'])
tfidf_transformer = TfidfTransformer().fit(text_bow)
text_tfidf = tfidf_transformer.transform(text_bow)

classifier_nb = MultinomialNB().fit(text_tfidf, data['Sentiment'])

sentiments = pd.DataFrame(columns=['text', 'class', 'prob'])
i = 0
for _, tweet in tweets.iterrows():
    i += 1
    try:
        bow_tweet = bow_transformer.transform(tweet)
        tfidf_tweet = tfidf_transformer.transform(bow_tweet)
        sentiments.loc[i-1, 'text'] = tweet.values[0]
        sentiments.loc[i-1, 'class'] = classifier_nb.predict(tfidf_tweet)[0]
        sentiments.loc[i-1, 'prob'] = classifier_nb.predict_proba(tfidf_tweet)[0][1]
    except Exception as e:
        sentiments.loc[i-1, 'text'] = tweet.values[0]

sentiments.to_csv('Sentiments.csv', encoding='utf-8')
sentiments.head()

C:\Users\Arunava\Anaconda3\lib\site-packages\sklearn\feature_extraction\text.py:502: UserWarning:

The parameter 'stop_words' will not be used since 'analyzer' != 'word'



,text,class,prob
0,call leader help protect refuge covid19 provid...,pos,0.796224
1,ogun state support cbn nirsal covid19 target c...,pos,0.727709
2,polic offici base namahadi polic station busi ...,pos,0.712461
3,taiwan vice presid chen chien jen countri figh...,neg,0.259388
4,break new york woman symptom die last week pre...,neg,0.196942


# **Sentiment Trend**

In [30]:
trend_df = pd.DataFrame()
trend_df["Date"] = new_sentiment_df['Date']
trend_df["Original_Tweet"] = original_df['original_text']
trend_df["Clean_Tweet"] = new_full_df['clean_tweet']
trend_df["Sentiment"] = sentiments['prob']
trend_df.to_csv('Sentiment Trend.csv', encoding='utf-8')
original_trend_df = trend_df
trend_df.head()

,Date,Original_Tweet,Clean_Tweet,Sentiment
0,2020-04-19,RT @GlblCtzn: .@priyankachopra is calling on l...,call leader help protect refuge covid19 provid...,0.796224
1,2020-04-19,RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,ogun state support cbn nirsal covid19 target c...,0.727709
2,2020-04-19,RT @AdvoBarryRoux: These 5 police officials ba...,polic offici base namahadi polic station busi ...,0.712461
3,2020-04-19,RT @MobilePunch: COVID-19: Oyo discharges two ...,covid19 oyo discharg two patient,0.259388
4,2020-04-19,My Condolences to the Family of those who did ...,condol famili surviv,0.196942


## **Overall Average Sentiment Trend**

### **Fine-grained Sentiment Classification**

In [31]:
trend_df["Refined_Sentiment"] = ""

for i in range(0,len(trend_df.index)):
    if trend_df['Sentiment'].iloc[i] >= 0.0 and trend_df['Sentiment'].iloc[i] < 0.25:
        trend_df['Refined_Sentiment'].iloc[i] = -1.0
    elif trend_df['Sentiment'].iloc[i] >= 0.25 and trend_df['Sentiment'].iloc[i] < 0.5:
        trend_df['Refined_Sentiment'].iloc[i] = -0.5
    elif trend_df['Sentiment'].iloc[i] == 0.5:
        trend_df['Refined_Sentiment'].iloc[i] = 0.0
    elif trend_df['Sentiment'].iloc[i] > 0.5 and trend_df['Sentiment'].iloc[i] < 0.75:
        trend_df['Refined_Sentiment'].iloc[i] = 0.5
    else:
        trend_df['Refined_Sentiment'].iloc[i] = 1.0

trend_df.to_csv('Overall Sentiment Trend.csv', encoding='utf-8')
trend_df.head()

,Date,Original_Tweet,Clean_Tweet,Sentiment,Refined_Sentiment
0,2020-04-19,RT @GlblCtzn: .@priyankachopra is calling on l...,call leader help protect refuge covid19 provid...,0.796224,1
1,2020-04-19,RT @OGSG_Official: OGUN STATE SUPPORT FOR CBN-...,ogun state support cbn nirsal covid19 target c...,0.727709,0.5
2,2020-04-19,RT @AdvoBarryRoux: These 5 police officials ba...,polic offici base namahadi polic station busi ...,0.712461,0.5
3,2020-04-19,RT @MobilePunch: COVID-19: Oyo discharges two ...,covid19 oyo discharg two patient,0.259388,-0.5
4,2020-04-19,My Condolences to the Family of those who did ...,condol famili surviv,0.196942,-1


### **Date-wise Overall Average Sentimet Trend**

In [32]:
avg_trend_df = pd.DataFrame()
avg_trend_df["Date"] = trend_df['Date']
avg_trend_df["Refined_Sentiment"] = trend_df['Refined_Sentiment']
avg_trend_df = avg_trend_df.set_index(['Date'])
avg_trend_df.Refined_Sentiment = avg_trend_df.Refined_Sentiment.astype('float64')
avg_trend_df = avg_trend_df.groupby('Date')['Refined_Sentiment'].mean().round(2)
avg_trend_df = pd.DataFrame(avg_trend_df)
avg_trend_df.to_csv('Overall Refined Sentiment Trend.csv', encoding='utf-8')
avg_trend_df.head()

,Refined_Sentiment
Date,
2020-04-19,-0.21
2020-04-22,-0.07
2020-04-23,-0.12
2020-04-24,0.12
2020-04-25,0.06


## **Average Sentiment Trend Shift Detection**

### **Sentiment Ratings Assignment**

In [33]:
trend_df = pd.DataFrame()
trend_df = original_trend_df

trend_df["Pos_Sentiment"] = ''
trend_df["Neg_Sentiment"] = ''

for i in range(0,len(trend_df.index)):
    if trend_df['Sentiment'].iloc[i] >= 0.0 and trend_df['Sentiment'].iloc[i] <= 0.50:
        trend_df['Neg_Sentiment'].iloc[i] = trend_df['Sentiment'].iloc[i].astype('float64')
    else:
        trend_df['Neg_Sentiment'].iloc[i] = 0.0

for i in range(0,len(trend_df.index)):
    if trend_df['Sentiment'].iloc[i] > 0.50 and trend_df['Sentiment'].iloc[i] < 1.0:
        trend_df['Pos_Sentiment'].iloc[i] = trend_df['Sentiment'].iloc[i].astype('float64')
    else:
        trend_df['Pos_Sentiment'].iloc[i] = 0.0

trend_df['Date'] = pd.to_datetime(trend_df['Date'])
trend_df.sort_values(by='Date', inplace=True)
trend_df.to_csv('Binary Sentiment Trend.csv', encoding='utf-8')
trend_df.head()

,Date,Original_Tweet,Clean_Tweet,Sentiment,Refined_Sentiment,Pos_Sentiment,Neg_Sentiment
0,2020-04-19,RT @GlblCtzn: .@priyankachopra is calling on l...,call leader help protect refuge covid19 provid...,0.796224,1,0.796224,0
16,2020-04-19,RT @OIC_IPHRC: 1/2 #OIC-IPHRC condemns the unr...,iphrc condemn unrel viciou campaign malign mus...,0.308438,-0.5,0,0.308438
15,2020-04-19,RT @JoeConchaTV: Wallace to Pelosi on her also...,wallac pelosi also downplay covid19 presid und...,0.282656,-0.5,0,0.282656
14,2020-04-19,RT @SaharaReporters: BREAKING: Jigawa Records ...,break jigawa record first covid19 case sahara ...,0.293104,-0.5,0,0.293104
12,2020-04-19,RT @KatiePhang: “‘[COVID-19] can attack almost...,covid19 attack almost anyth bodi devast conseq...,0.750349,1,0.750349,0


### **Date-wise Average Sentiment Trend Shift**

In [34]:
avg_trend_df = pd.DataFrame()
avg_trend_df["Date"] = trend_df['Date']
avg_trend_df["Avg_Pos_Sentiment"] = trend_df['Pos_Sentiment'].astype('float64')
avg_trend_df["Avg_Neg_Sentiment"] = trend_df['Neg_Sentiment'].astype('float64')

avg_trend_df['Date'] = pd.to_datetime(avg_trend_df['Date'])
avg_trend_df = avg_trend_df.set_index(['Date'])
avg_trend_df = avg_trend_df.groupby(pd.Grouper(freq='1d')).mean()
avg_trend_df = avg_trend_df.dropna()
avg_trend_df.to_csv('Binary Refined Sentiment Trend.csv', encoding='utf-8')
avg_trend_df.head()

,Avg_Pos_Sentiment,Avg_Neg_Sentiment
Date,,
2020-04-19,0.264747,0.165383
2020-04-22,0.361192,0.136890
2020-04-23,0.310630,0.148877
2020-04-24,0.463307,0.085823
2020-04-25,0.400304,0.121273


# **Prepare Final Data**

In [35]:
cols = ['Sentiment', 'Text']
final_data_df = pd.DataFrame(columns=cols, index=range(len(sentiments.index)))
final_data_df["Sentiment"] = ""

for i in range(0,len(sentiments.index)):
    if sentiments['class'].iloc[i] == 'neg':
        final_data_df['Sentiment'].iloc[i] = 0
    else:
        final_data_df['Sentiment'].iloc[i] = 1

final_data_df["Text"] = sentiments['text']
final_data_df.to_csv("Tweet Final Data.csv", index=False)
final_data_df.head(5)

,Sentiment,Text
0,1,call leader help protect refuge covid19 provid...
1,1,ogun state support cbn nirsal covid19 target c...
2,1,polic offici base namahadi polic station busi ...
3,0,taiwan vice presid chen chien jen countri figh...
4,0,break new york woman symptom die last week pre...


## **Prepare Data for Vectorization**

In [36]:
def ingest():
    data = pd.read_csv('Tweet Final Data.csv')
    data = data[data.Sentiment.isnull() == False]
    data['Sentiment'] = data['Sentiment'].map(int)
    data = data[data['Text'].isnull() == False]
    data.reset_index(inplace=True)
    data.drop('index',axis=1,inplace=True)
    print ('dataset loaded with shape', data.shape)    
    return data

In [37]:
data = ingest()
data.head(5)

dataset loaded with shape (976, 2)


,Sentiment,Text
0,1,call leader help protect refuge covid19 provid...
1,1,ogun state support cbn nirsal covid19 target c...
2,1,polic offici base namahadi polic station busi ...
3,0,taiwan vice presid chen chien jen countri figh...
4,0,break new york woman symptom die last week pre...


In [38]:
X = data['Text']
Y = to_categorical(data['Sentiment'].values)
print(Y)

[[0. 1.]
 [0. 1.]
 [0. 1.]
 ...
 [0. 1.]
 [0. 1.]
 [1. 0.]]


In [39]:
x_train, x_test, y_train, y_test = train_test_split(X, Y, test_size=0.2)

## **Total number of words/features**

In [40]:
all_words=' '.join(x_train)
all_words=word_tokenize(all_words)
#print(all_words)
dist=FreqDist(all_words)

num_unique_word=len(dist)
num_unique_word

2945

## **Number of words for each phrase/text**

In [41]:
tweet_len=[]
for text in x_train:
    word=word_tokenize(text)
#     print(text)
    l=len(word)
    tweet_len.append(l)
    
max_tweet_len=np.max(tweet_len)
max_tweet_len

20

In [42]:
max_features = num_unique_word
max_words = max_tweet_len

## **Tokenizing the words**

In [43]:
tokenizer = Tokenizer(num_words=max_features)
tokenizer.fit_on_texts(x_train)
x_train = tokenizer.texts_to_sequences(x_train)
x_test = tokenizer.texts_to_sequences(x_test)

## **Sequence Padding**

In [44]:
x_train = sequence.pad_sequences(x_train, maxlen=max_words)
x_test = sequence.pad_sequences(x_test, maxlen=max_words)

## **Classification Report & Confusion Matrix**

In [45]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import classification_report, confusion_matrix

# Create DecisionTree model
classifier = DecisionTreeClassifier()

# Train model and make predictions
y_pred = classifier.fit(x_train, y_train).predict(x_test)

# Create a classification report
print()
print("\t\t: Classification Report :\t\t")
print("\t\t=========================\t\t")
print()
print(classification_report(y_test, y_pred))

# Create a confusion matrix
print()
print("\t\t: Confusion Matrix :\t\t")
print("\t\t====================\t\t")
print()
print(confusion_matrix(y_test.argmax(axis=1), y_pred.argmax(axis=1)))


		: Classification Report :		

              precision    recall  f1-score   support

           0       0.51      0.49      0.50        99
           1       0.49      0.51      0.50        97

   micro avg       0.50      0.50      0.50       196
   macro avg       0.50      0.50      0.50       196
weighted avg       0.50      0.50      0.50       196
 samples avg       0.50      0.50      0.50       196


		: Confusion Matrix :		

[[49 50]
 [48 49]]


# **Hybrid Convolutional LSTM**

# **CNN + Sequential LSTM**

## **Define the First Hybrid Model**

In [46]:
model1= Sequential()
model1.add(Embedding(max_features,100,input_length=max_words))
model1.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model1.add(MaxPooling1D(pool_size=2))
model1.add(Dropout(0.25))
model1.add(LSTM(128,return_sequences=True))
model1.add(Dropout(0.2))
model1.add(Flatten())
model1.add(Dense(128,activation='relu'))
model1.add(Dropout(0.1))
model1.add(Dense(64,activation='relu'))
model1.add(Dense(32,activation='relu'))
model1.add(Dense(2,activation='sigmoid'))
model1.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model1.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 100)           294500    
_________________________________________________________________
conv1d (Conv1D)              (None, 20, 64)            19264     
_________________________________________________________________
max_pooling1d (MaxPooling1D) (None, 10, 64)            0         
_________________________________________________________________
dropout (Dropout)            (None, 10, 64)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 10, 128)           98816     
_________________________________________________________________
dropout_1 (Dropout)          (None, 10, 128)           0         
_________________________________________________________________
flatten (Flatten)            (None, 1280)              0

## **Train the CNN + Seq-LSTM Model**

In [47]:
history = model1.fit(x_train, y_train, epochs=6, validation_data=(x_test, y_test), batch_size=32, verbose=2)

Epoch 1/6
25/25 - 4s - loss: 0.6941 - accuracy: 0.4705 - val_loss: 0.6931 - val_accuracy: 0.5051
Epoch 2/6
25/25 - 0s - loss: 0.6903 - accuracy: 0.5244 - val_loss: 0.6841 - val_accuracy: 0.5255
Epoch 3/6
25/25 - 0s - loss: 0.4415 - accuracy: 0.8487 - val_loss: 0.6160 - val_accuracy: 0.7602
Epoch 4/6
25/25 - 0s - loss: 0.0727 - accuracy: 0.9744 - val_loss: 0.7153 - val_accuracy: 0.7653
Epoch 5/6
25/25 - 0s - loss: 0.0095 - accuracy: 0.9987 - val_loss: 0.8137 - val_accuracy: 0.7959
Epoch 6/6
25/25 - 0s - loss: 0.0041 - accuracy: 0.9987 - val_loss: 0.8947 - val_accuracy: 0.8010


### **Final Validation Accuracy of CNN + Seq-LSTM Model**

In [48]:
score = model1.evaluate(x_test, y_test, batch_size=32, verbose=2)
print (score[1])

7/7 - 0s - loss: 0.8947 - accuracy: 0.8010
0.8010203838348389


# **CNN + Bidirectional LSTM**

## **Define the Second Hybrid Model**

In [49]:
model2= Sequential()
model2.add(Embedding(max_features,100,input_length=max_words))
model2.add(Conv1D(64,kernel_size=3,padding='same',activation='relu'))
model2.add(MaxPooling1D(pool_size=2))
model2.add(Dropout(0.25))
model2.add(Bidirectional(LSTM(128,return_sequences=True)))
model2.add(Dropout(0.2))
model2.add(Flatten())
model2.add(Dense(128,activation='relu'))
model2.add(Dropout(0.1))
model2.add(Dense(64,activation='relu'))
model2.add(Dense(32,activation='relu'))
model2.add(Dense(2,activation='sigmoid'))
model2.compile(loss='binary_crossentropy',optimizer=Adam(lr=0.001),metrics=['accuracy'])
model2.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 20, 100)           294500    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 20, 64)            19264     
_________________________________________________________________
max_pooling1d_1 (MaxPooling1 (None, 10, 64)            0         
_________________________________________________________________
dropout_3 (Dropout)          (None, 10, 64)            0         
_________________________________________________________________
bidirectional (Bidirectional (None, 10, 256)           197632    
_________________________________________________________________
dropout_4 (Dropout)          (None, 10, 256)           0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 2560)             

## **Train the CNN + Bi-LSTM Model**

In [50]:
history = model2.fit(x_train, y_train, epochs=6, validation_data=(x_test, y_test), batch_size=32, verbose=2)

Epoch 1/6
25/25 - 3s - loss: 0.6936 - accuracy: 0.5064 - val_loss: 0.6907 - val_accuracy: 0.5051
Epoch 2/6
25/25 - 0s - loss: 0.5949 - accuracy: 0.6410 - val_loss: 0.5830 - val_accuracy: 0.7806
Epoch 3/6
25/25 - 0s - loss: 0.1209 - accuracy: 0.9679 - val_loss: 0.5961 - val_accuracy: 0.7551
Epoch 4/6
25/25 - 0s - loss: 0.0175 - accuracy: 0.9962 - val_loss: 0.7607 - val_accuracy: 0.7857
Epoch 5/6
25/25 - 0s - loss: 6.6369e-04 - accuracy: 1.0000 - val_loss: 0.9304 - val_accuracy: 0.7704
Epoch 6/6
25/25 - 0s - loss: 4.1419e-04 - accuracy: 1.0000 - val_loss: 1.0957 - val_accuracy: 0.7806


### **Final Validation Accuracy of CNN + Bi-LSTM Model**

In [51]:
score = model2.evaluate(x_test, y_test, batch_size=32, verbose=2)
print (score[1])

7/7 - 0s - loss: 1.0957 - accuracy: 0.7806
0.7806122303009033
